In [ ]:
import os
import cv2
import torch
import matplotlib
import numpy as np
import torch.nn.functional as F
from torchvision.transforms import Compose


In [ ]:
from fish_size.depth_anything.dpt import DepthAnything
from fish_size.depth_anything.util.transform import Resize, NormalizeImage, PrepareForNet

from fish_size.depth_anything_v2.dpt import DepthAnythingV2


In [ ]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'


In [ ]:
model_configs = {
    'vits': {'encoder': 'vits', 'features': 64, 'out_channels': [48, 96, 192, 384]},
    'vitb': {'encoder': 'vitb', 'features': 128, 'out_channels': [96, 192, 384, 768]},
    'vitl': {'encoder': 'vitl', 'features': 256, 'out_channels': [256, 512, 1024, 1024]},
    'vitg': {'encoder': 'vitg', 'features': 384, 'out_channels': [1536, 1536, 1536, 1536]}
}


In [ ]:
da = DepthAnything.from_pretrained('LiheYoung/depth_anything_{}14'.format('vitl')).to(DEVICE).eval()


In [ ]:
da2 = DepthAnythingV2(**model_configs['vitl'])
da2.load_state_dict(torch.load(f'./fish_size/checkpoints/depth_anything_v2_{"vitl"}.pth', map_location='cpu'))
da2 = da2.to(DEVICE).eval()


In [ ]:
transform = Compose([
    Resize(
        width=518,
        height=518,
        resize_target=False,
        keep_aspect_ratio=True,
        ensure_multiple_of=14,
        resize_method='lower_bound',
        image_interpolation_method=cv2.INTER_CUBIC,
    ),
    NormalizeImage(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    PrepareForNet(),
])


In [ ]:
image = cv2.imread('dataset/images/test/objt_rs.JPG')


In [ ]:
rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) / 255.0

h, w = rgb_image.shape[:2]

rgb_image = transform({'image': rgb_image})['image']
rgb_image = torch.from_numpy(rgb_image).unsqueeze(0).to(DEVICE)

with torch.no_grad():
    depth_da = da(rgb_image)

depth_da = F.interpolate(depth_da[None], (h, w), mode='bilinear', align_corners=False)[0, 0]
depth_da = (depth_da - depth_da.min()) / (depth_da.max() - depth_da.min()) * 255.0

depth_da = depth_da.cpu().numpy().astype(np.uint8)


In [ ]:
depth_da2 = da2.infer_image(image, 518)

depth_da2 = (depth_da2 - depth_da2.min()) / (depth_da2.max() - depth_da2.min()) * 255.0
depth_da2 = depth_da2.astype(np.uint8)


In [ ]:
depth_da_g = np.repeat(depth_da[..., np.newaxis], 3, axis=-1)
depth_da2_g = np.repeat(depth_da2[..., np.newaxis], 3, axis=-1)


In [ ]:
cmap = matplotlib.colormaps.get_cmap('Spectral_r')


In [ ]:
depth_da_c_l = (cmap(depth_da)[:, :, :3] * 255)[:, :, ::-1].astype(np.uint8)
depth_da2_c_l = (cmap(depth_da2)[:, :, :3] * 255)[:, :, ::-1].astype(np.uint8)


In [ ]:
def imageWrite(image1, image2, filename):
    split_region = np.ones((image1.shape[0], 50, 3), dtype=np.uint8) * 255
    combined_result = cv2.hconcat([image1, split_region, image2])

    cv2.imwrite(os.path.join('./runs/vis_depth', os.path.splitext(os.path.basename(filename))[0] + '.png'), combined_result)


In [ ]:
imageWrite(depth_da_g, depth_da_c_l, 'DA_L')
imageWrite(depth_da2_g, depth_da2_c_l, "DA2_L")


In [ ]:
import os
import random

# 베이스 디렉토리 설정
base_dir = 'dataset/images'

# 하위 폴더 목록
subfolders = ['bp', 'kr', 'of', 'rb', 'rs']

# 각 폴더에서 읽을 이미지 수
images_per_folder = 50

# 이미지 경로를 저장할 리스트
image_paths = []

# 각 하위 폴더 순회
for subfolder in subfolders:
    folder_path = os.path.join(base_dir, subfolder)
    # 이미지 파일 목록 가져오기
    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif'))]
    # 이미지 파일이 20개 이상인지 확인
    if len(image_files) >= images_per_folder:
        # 20개의 이미지를 무작위로 선택
        selected_images = random.sample(image_files, images_per_folder)
        # 선택된 이미지의 전체 경로를 리스트에 추가
        for image_file in selected_images:
            image_path = os.path.join(folder_path, image_file)
            image_paths.append(image_path)
    else:
        # 이미지가 20장 미만인 경우 폴더를 건너뜀
        print(f"폴더 '{subfolder}'는 이미지가 {images_per_folder}장 미만이므로 건너뜁니다.")

# 결과 확인 (선택 사항)
print(f"총 {len(image_paths)}개의 이미지 경로가 저장되었습니다.")


In [ ]:
from time import perf_counter

start = perf_counter()

for i in image_paths:
    image = cv2.imread(i)

    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) / 255.0

    h, w = rgb_image.shape[:2]

    rgb_image = transform({'image': rgb_image})['image']
    rgb_image = torch.from_numpy(rgb_image).unsqueeze(0).to(DEVICE)

    with torch.no_grad():
        depth_da = da(rgb_image)

    depth_da = F.interpolate(depth_da[None], (h, w), mode='bilinear', align_corners=False)[0, 0]
    depth_da = (depth_da - depth_da.min()) / (depth_da.max() - depth_da.min()) * 255.0

    depth_da = depth_da.cpu().numpy().astype(np.uint8)

end = perf_counter()
print(f'Elapsed time: {end - start}')


In [ ]:
start = perf_counter()

for i in image_paths:
    image = cv2.imread(i)

    depth_da2 = da2.infer_image(image, 518)

    depth_da2 = (depth_da2 - depth_da2.min()) / (depth_da2.max() - depth_da2.min()) * 255.0
    depth_da2 = depth_da2.astype(np.uint8)

end = perf_counter()
print(f'Elapsed time: {end - start}')


In [ ]:
da = DepthAnything.from_pretrained('LiheYoung/depth_anything_{}14'.format('vits')).to(DEVICE).eval()


In [ ]:
da2 = DepthAnythingV2(**model_configs['vits'])
da2.load_state_dict(torch.load(f'./fish_size/checkpoints/depth_anything_v2_{"vits"}.pth', map_location='cpu'))
da2 = da2.to(DEVICE).eval()


In [ ]:
rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) / 255.0

h, w = rgb_image.shape[:2]

rgb_image = transform({'image': rgb_image})['image']
rgb_image = torch.from_numpy(rgb_image).unsqueeze(0).to(DEVICE)

with torch.no_grad():
    depth_da = da(rgb_image)

depth_da = F.interpolate(depth_da[None], (h, w), mode='bilinear', align_corners=False)[0, 0]
depth_da = (depth_da - depth_da.min()) / (depth_da.max() - depth_da.min()) * 255.0

depth_da = depth_da.cpu().numpy().astype(np.uint8)


In [ ]:
depth_da2 = da2.infer_image(image, 518)

depth_da2 = (depth_da2 - depth_da2.min()) / (depth_da2.max() - depth_da2.min()) * 255.0
depth_da2 = depth_da2.astype(np.uint8)


In [ ]:
depth_da_g = np.repeat(depth_da[..., np.newaxis], 3, axis=-1)
depth_da2_g = np.repeat(depth_da2[..., np.newaxis], 3, axis=-1)


In [ ]:
cmap = matplotlib.colormaps.get_cmap('Spectral_r')


In [ ]:
depth_da_c_s = (cmap(depth_da)[:, :, :3] * 255)[:, :, ::-1].astype(np.uint8)
depth_da2_c_s = (cmap(depth_da2)[:, :, :3] * 255)[:, :, ::-1].astype(np.uint8)


In [ ]:
imageWrite(depth_da_g, depth_da_c_l, 'DA_S')
imageWrite(depth_da2_g, depth_da2_c_l, "DA2_S")


In [ ]:
imageWrite(depth_da_c_s, depth_da_c_l, 'DA')
imageWrite(depth_da2_c_s, depth_da2_c_l, "DA2")


In [ ]:
start = perf_counter()

for i in image_paths:
    image = cv2.imread(i)

    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) / 255.0

    h, w = rgb_image.shape[:2]

    rgb_image = transform({'image': rgb_image})['image']
    rgb_image = torch.from_numpy(rgb_image).unsqueeze(0).to(DEVICE)

    with torch.inference_mode():
        depth_da = da(rgb_image)

    depth_da = F.interpolate(depth_da[None], (h, w), mode='bilinear', align_corners=False)[0, 0]
    depth_da = (depth_da - depth_da.min()) / (depth_da.max() - depth_da.min()) * 255.0

    depth_da = depth_da.cpu().numpy().astype(np.uint8)

end = perf_counter()
print(f'Elapsed time: {end - start}')


In [ ]:
start = perf_counter()

for i in image_paths:
    image = cv2.imread(i)

    depth_da2 = da2.infer_image(image, 518)

    depth_da2 = (depth_da2 - depth_da2.min()) / (depth_da2.max() - depth_da2.min()) * 255.0
    depth_da2 = depth_da2.astype(np.uint8)

end = perf_counter()
print(f'Elapsed time: {end - start}')
